# Mounts and import

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U kaleido

In [ ]:
from tqdm.notebook import tqdm
import pickle
import os
import numpy as np
import gc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
rename_dict = {'bottle': 'Flasche', 
               'cable': 'Kabel', 
               'capsule': 'Kapsel',
               'carpet': 'Teppich', 
               'grid': 'Netz', 
               'hazelnut': 'Haselnuss', 
               'leather': 'Leder', 
               'metal_nut': 'Metall-Mutter', 
               'pill': 'Pille', 
               'screw': 'Schraube', 
               'tile': 'Fliese', 
               'toothbrush': 'Zahnbürste',
               'transistor': 'Transistor', 
               'wood': 'Holz', 
               'zipper': 'Reißverschluss'}


_to_test = [
            'bottle', 'cable', 'capsule',
            'carpet', 'grid', 'hazelnut',
            'leather', 'metalnut', 'pill',
            'screw', 'tile', 'toothbrush',
            'transistor', 'wood', 'zipper'
            ]
            
def extract_results(save_path):
  results = {}
  for category in tqdm(_to_test):
    if 'b3' in save_path:
      end = 8
    elif 'b7' in save_path:
      end = 9
    else:
      print(save_path, 'NOT FOUND')
      break

    results[category] = {}

    for layer in range(1, end):

      # create results folder
      results_save_path = f'{save_path}/{category}/results_layer_{layer}.pickle'
      with open(results_save_path, 'rb') as f:
        p = pickle.load(f)
        results[category][layer] = p 
  return results

# Efficientnet - Graphs


In [ ]:


def plot_figure_master(save_path, name):
  results = extract_results(save_path)

  results_flattened = {}
  for i in results:
    results_flattened[i] = {}
    for j in results[i]:
      for k in results[i][j]:
        results_flattened[i][f'{j}_{k}'] = results[i][j][k]

  results_reversed = {}
  for i in results_flattened:
    for j in results_flattened[i]:
      if j not in results_reversed:
        results_reversed[j] = {}
      results_reversed[j][i] = results_flattened[i][j]

  for_df = {'labels': _to_test}
  for i in results_reversed:
    for_df[i] = []
    for l in for_df['labels']:
      for_df[i].append(results_reversed[i][l])
  df = pd.DataFrame(for_df).set_index('labels')
  df.loc['mean'] = df.mean()
  df.loc[:, 'mean'] = df.mean(axis=1)
  df.sort_values(by ='mean', axis=1, ascending=False, inplace=True)
  df.sort_values(by ='mean', axis=0, ascending=False, inplace=True)

  plt.figure(figsize=(30,50))
  
  sns.set(font_scale=1.8)
  g = sns.heatmap(df.T, annot=True, cmap="YlGnBu")
  
  g._axes.set_xticklabels(g._axes.get_xmajorticklabels(), fontsize = 22)
  g._axes.set_yticklabels(g._axes.get_ymajorticklabels(), fontsize = 22)
  plt.xticks(rotation=30)
  folder = name.split(' - ')[0].replace(' ', '_').lower()
  folder_path = f'drive/MyDrive/data/efficientnet_graphics/{folder}'
  if not os.path.isdir(folder_path):
    os.mkdir(folder_path)
  plt.xlabel('')
  figure_name = 'complete_' + name.replace('-', '').replace(' ', '_').replace('__', '_').lower()
  plt.tight_layout()
  plt.savefig(f'{folder_path}/{figure_name}.png', dpi=300)
  plt.show()
  return df

In [ ]:
# Efficient Net B3 

save_path = 'drive/MyDrive/data/efficientnet_b3_results/model_eval'
name = 'EfficientNet B3 - Image Level'
df_b3_img = plot_figure_master(save_path, name)

save_path = 'drive/MyDrive/data/efficientnet_b3_results/model_eval_patchlevel'
name = 'EfficientNet B3 - Patch Level'
df_b3_patch = plot_figure_master(save_path, name)

# Efficient Net B7 
save_path = 'drive/MyDrive/data/efficientnet_b7_results/model_eval'
name = 'EfficientNet B7 - Image Level'
df_b7_img = plot_figure_master(save_path, name)

save_path = 'drive/MyDrive/data/efficientnet_b7_results/model_eval_patchlevel'
name = 'EfficientNet B7 - Patch Level'
df_b7_patch = plot_figure_master(save_path, name)

## Plot best performance per EfficientNet and per img/patch approach

In [ ]:
df_b3_img.rename(columns = {c : f'B3_{c}_img' for c in df_b3_img.columns}, inplace=True)
df_b3_patch.rename(columns = {c : f'B3_{c}_patch' for c in df_b3_patch.columns}, inplace=True)
df_b7_img.rename(columns = {c : f'B7_{c}_img' for c in df_b7_img.columns}, inplace=True)
df_b7_patch.rename(columns = {c : f'B7_{c}_patch' for c in df_b7_patch.columns}, inplace=True)

In [ ]:
patch_results_save_path = f'drive/MyDrive/data/wideresnet_evaluation/results_patchcore.pickle'
with open(patch_results_save_path, 'rb') as f:
  p = pickle.load(f)

nn = p['bottle'].keys()

flatten_p = {'categories': []}
for n in nn:
  flatten_p[n] = []

for k, v in p.items():
  k1 = k if 'metal' not in k else 'metalnut'
  flatten_p['categories'].append(k1)
  for n in nn:
    flatten_p[n].append(v[n])
df_wide = pd.DataFrame(flatten_p).set_index('categories')

df_wide.loc['mean', :] = df_wide.mean(axis=0)
df_wide.sort_values(by='mean', axis=1, inplace=True, ascending=False)
df_wide.rename(columns = {c : f'WideResnet_n{c}' for c in df_wide.columns}, inplace=True)

In [ ]:
df_plot = pd.concat([
                     df_b3_img.iloc[:, 0],
                     df_b3_patch.iloc[:, 0],
                     df_b7_img.iloc[:, 0],
                     df_b7_patch.iloc[:, 0],
                     df_wide.iloc[:, 0]
                     ],
                    axis=1
                    ).T
df_plot.drop(columns=['mean'], inplace=True)
df_plot['mean'] = df_plot.mean(axis=1)
df_plot.loc['mean', :] = df_plot.mean(axis=0)
df_plot.sort_values(by='mean', ascending=False, inplace=True)
df_plot.sort_values(by='mean', axis=1, ascending=False, inplace=True)

df_plot.drop(columns=['mean'], inplace=True)
df_plot.drop(index=['mean'], inplace=True)
df_plot['mean'] = df_plot.mean(axis=1)
df_plot.loc['mean', :] = df_plot.mean(axis=0)

plt.figure(figsize=(14,4))
plt.title('Besten Algorithmuskombinationen', fontdict={'fontsize': 20}, loc='left')

ax = sns.heatmap(df_plot, annot=True, cmap="YlGnBu", fmt='.3f', linewidths=.1)
#plt.ylabel('Algorithmus - Kombination', fontsize=14)
plt.xlabel('Kategorie', fontsize=14)
plt.xticks(rotation=30)
plt.tight_layout()
plt.savefig(f'drive/MyDrive/data/efficientnet_graphics/best_algorithm_comparison.png', dpi=400)

# Plot figures per Category

In [ ]:
def plot_figure_per_category(save_path, name):
  results = extract_results(save_path)
  v_mean = []
  for c in results:
    data = {'Schicht': []}
    for l in results[c]:
      data['Schicht'].append(l)
      if l == 1 and 'patchlevel' not in save_path:
        data['AutoEncoder'] = [np.nan]
        data['DeepSVDD'] = [np.nan]
      for a in results[c][l]:
        if a not in data:
          data[a] = []
        data[a].append(results[c][l][a])

    df = pd.DataFrame(data).set_index('Schicht')
    df = df.reindex(sorted(df.columns), axis=1)
    df = df.reindex(sorted(df.index)[::-1], axis=0)

    plt.figure(figsize=(16,9))
    plt.title(name + ' - ' + c.capitalize(), fontdict={'fontsize': 25}, loc='left')
    ax = sns.heatmap(df, annot=True, cmap="YlGnBu", annot_kws={"fontsize":18})

    ax._axes.set_xticklabels(ax._axes.get_xmajorticklabels(), fontsize = 16)
    ax._axes.set_yticklabels(ax._axes.get_ymajorticklabels(), fontsize = 16)
    ax.collections[0].colorbar.ax.tick_params(labelsize=16)

    plt.ylabel('Schicht', fontsize=18)
    plt.xlabel('Algorithmus', fontsize=18)
    ax.yaxis.set_label_position("right")


    folder = name.split(' - ')[0].replace(' ', '_').lower()
    folder_path = f'drive/MyDrive/data/efficientnet_graphics/{folder}/per_category'
    if not os.path.isdir(folder_path):
      os.mkdir(folder_path)
    folder_path = folder_path + f'/{name.split(" - ")[1].replace(" ", "_").lower()}'
    if not os.path.isdir(folder_path):
      os.mkdir(folder_path)

    figure_name = c + '_' + name.replace('-', '').replace(' ', '_').replace('__', '_').lower()
    plt.tight_layout()
    plt.savefig(f'{folder_path}/{figure_name}.png', dpi=400)
    plt.close()
  return df

In [ ]:
# Efficient Net B3 
save_path = 'drive/MyDrive/data/efficientnet_b3_results/model_eval'
name = 'EfficientNet B3 - Image Level'
df_b3_img = plot_figure_per_category(save_path, name)

save_path = 'drive/MyDrive/data/efficientnet_b3_results/model_eval_patchlevel'
name = 'EfficientNet B3 - Patch Level'
df_b3_patch = plot_figure_per_category(save_path, name)

# Efficient Net B7 
save_path = 'drive/MyDrive/data/efficientnet_b7_results/model_eval'
name = 'EfficientNet B7 - Image Level'
df_b7_img = plot_figure_per_category(save_path, name)

save_path = 'drive/MyDrive/data/efficientnet_b7_results/model_eval_patchlevel'
name = 'EfficientNet B7 - Patch Level'
df_b7_patch = plot_figure_per_category(save_path, name)

## Average

In [ ]:
def plot_figure_per_category_avg(save_path, name):
  results = extract_results(save_path)
  v_mean = []
  for c in results:
    data = {'Schicht': []}
    for l in results[c]:
      data['Schicht'].append(l)
      if l == 1 and 'patchlevel' not in save_path:
        data['AutoEncoder'] = [np.nan]
        data['DeepSVDD'] = [np.nan]
      for a in results[c][l]:
        if a not in data:
          data[a] = []
        data[a].append(results[c][l][a])

    df = pd.DataFrame(data).set_index('Schicht')
    df = df.reindex(sorted(df.columns), axis=1)
    df = df.reindex(sorted(df.index)[::-1], axis=0)
    v_mean.append(df.values)

  a = np.dstack(v_mean)
  c = []
  for i in range(a.shape[0]):
    c.append(np.nanmean(a[i,:,:], axis=1))
  df1 = pd.DataFrame(np.array(c), columns=df.columns.tolist(), index=df.index.tolist())
  
  df1.loc['mean'] = df1.mean()
  df1.loc[:, 'mean'] = df1.mean(axis=1)
  df1.sort_values(by ='mean', axis=1, ascending=False, inplace=True)
  df1.drop(columns=['mean'], inplace=True)
  df1.drop(index=['mean'], inplace=True)
  df1.loc['mean'] = df1.mean()
  df1.loc[:, 'mean'] = df1.mean(axis=1)

  plt.figure(figsize=(16,9))
  plt.title(name + ' - Algorithmus Durchschnitt', fontdict={'fontsize': 25}, loc='left')
  ax = sns.heatmap(df1, annot=True, cmap="YlGnBu", annot_kws={"fontsize":18})

  ax._axes.set_xticklabels(ax._axes.get_xmajorticklabels(), fontsize = 16)
  ax._axes.set_yticklabels(ax._axes.get_ymajorticklabels(), fontsize = 16)
  ax.collections[0].colorbar.ax.tick_params(labelsize=16)

  plt.xticks(rotation=30)
  plt.ylabel('Schicht', fontsize=18)
  plt.xlabel('Algorithmus', fontsize=18)
  ax.yaxis.set_label_position("right")

  folder = name.split(' - ')[0].replace(' ', '_').lower()
  folder_path = f'drive/MyDrive/data/efficientnet_graphics/{folder}/per_category'
  if not os.path.isdir(folder_path):
    os.mkdir(folder_path)

  figure_name = 'category_avg_' + name.replace('-', '').replace(' ', '_').replace('__', '_').lower()
  plt.tight_layout()
  plt.savefig(f'{folder_path}/{figure_name}.png', dpi=400)


In [ ]:
# EfficientNet B3
save_path = 'drive/MyDrive/data/efficientnet_b3_results/model_eval_patchlevel'
name = 'EfficientNet B3 - Patch Level'
plot_figure_per_category_avg(save_path, name)

save_path = 'drive/MyDrive/data/efficientnet_b3_results/model_eval'
name = 'EfficientNet B3 - Image Level'
plot_figure_per_category_avg(save_path, name)

# EfficientNet B7
save_path = 'drive/MyDrive/data/efficientnet_b7_results/model_eval_patchlevel'
name = 'EfficientNet B7 - Patch Level'
plot_figure_per_category_avg(save_path, name)

save_path = 'drive/MyDrive/data/efficientnet_b7_results/model_eval'
name = 'EfficientNet B7 - Image Level'
plot_figure_per_category_avg(save_path, name)

# Plot figures per Layer

In [ ]:
def plot_figure_per_layer(save_path, name):
  results = extract_results(save_path)

  if 'b3' in save_path:
    end = 8
  elif 'b7' in save_path:
    end = 9
  else:
    print(save_path, 'NOT FOUND')
    return

  for l in range(1, end):
    data = {'Kategorie': []}
    for c in results:
      data['Kategorie'].append(c)
      for a in results[c][l]:
        if a not in data:
          data[a] = []
        data[a].append(results[c][l][a])

    df = pd.DataFrame(data)
    if l == 1:
      df['AutoEncoder'] = np.NaN
      df['DeepSVDD'] = np.NaN
    df.set_index('Kategorie', inplace=True)
    #df.loc['mean'] = df.mean()
    #df.loc[:, 'mean'] = df.mean(axis=1)
    #df.sort_values(by='mean', axis=0, ascending=False, inplace=True)
    df = df.reindex(sorted(df.columns), axis=1)
    df = df.reindex(sorted(df.index), axis=0)
    #df.sort_values(by=[7, 6, 5, 4, 3], axis=1, ascending=False, inplace=True)

    plt.figure(figsize=(16,9))
    plt.title(name + ' - Schicht: ' + str(l), fontdict={'fontsize': 25}, loc='left')
    ax = sns.heatmap(df, annot=True, cmap="YlGnBu", annot_kws={"fontsize":17})

    ax._axes.set_xticklabels(ax._axes.get_xmajorticklabels(), fontsize = 16)
    ax._axes.set_yticklabels(ax._axes.get_ymajorticklabels(), fontsize = 16)
    
    ax.collections[0].colorbar.ax.tick_params(labelsize=16)

    plt.xticks(rotation=30)
    plt.ylabel('Kategorie', fontsize=18)
    plt.xlabel('Algorithmus', fontsize=18)
    ax.yaxis.set_label_position("right")

    folder = name.split(' - ')[0].replace(' ', '_').lower()
    folder_path = f'drive/MyDrive/data/efficientnet_graphics/{folder}/per_layer'
    if not os.path.isdir(folder_path):
      os.mkdir(folder_path)
    folder_path = folder_path + f'/{name.split(" - ")[1].replace(" ", "_").lower()}'
    if not os.path.isdir(folder_path):
      os.mkdir(folder_path)

    figure_name = 'layer_' + str(l) + '_' + name.replace('-', '').replace(' ', '_').replace('__', '_').lower()
    plt.tight_layout()
    plt.savefig(f'{folder_path}/{figure_name}.png', dpi=400)
    plt.close()


In [ ]:
# Efficient Net B3
save_path = 'drive/MyDrive/data/efficientnet_b3_results/model_eval'
name = 'EfficientNet B3 - Image Level'
v_mean = plot_figure_per_layer(save_path, name)

save_path = 'drive/MyDrive/data/efficientnet_b3_results/model_eval_patchlevel'
name = 'EfficientNet B3 - Patch Level'
plot_figure_per_layer(save_path, name)

# Efficient Net B7 
save_path = 'drive/MyDrive/data/efficientnet_b7_results/model_eval'
name = 'EfficientNet B7 - Image Level'
v_mean = plot_figure_per_layer(save_path, name)

save_path = 'drive/MyDrive/data/efficientnet_b7_results/model_eval_patchlevel'
name = 'EfficientNet B7 - Patch Level'
plot_figure_per_layer(save_path, name)

## Average

In [ ]:
def plot_figure_per_layer_avg(save_path, name):
  results = extract_results(save_path)

  if 'b3' in save_path:
    end = 8
  elif 'b7' in save_path:
    end = 9
  else:
    print(save_path, 'NOT FOUND')
    return

  v_mean = []

  for l in range(1, end):
    data = {'Kategorie': []}
    for c in results:
      data['Kategorie'].append(c)
      for a in results[c][l]:
        if a not in data:
          data[a] = []
        data[a].append(results[c][l][a])

    df = pd.DataFrame(data)
    if l == 1:
      df['AutoEncoder'] = np.NaN
      df['DeepSVDD'] = np.NaN
    df.set_index('Kategorie', inplace=True)
    df = df.reindex(sorted(df.columns), axis=1)
    df = df.reindex(sorted(df.index), axis=0)
    v_mean.append(df.values)
  

  a = np.dstack(v_mean)
  c = []
  for i in range(a.shape[0]):
    c.append(np.nanmean(a[i,:,:], axis=1))
  df1 = pd.DataFrame(np.array(c), columns=df.columns.tolist(), index=df.index.tolist())

  
  df1.loc['mean'] = df1.mean()
  df1.loc[:, 'mean'] = df1.mean(axis=1)
  df1.sort_values(by ='mean', axis=1, ascending=False, inplace=True)
  df1.drop(columns=['mean'], inplace=True)
  df1.drop(index=['mean'], inplace=True)
  df1.loc['mean'] = df1.mean()
  df1.loc[:, 'mean'] = df1.mean(axis=1)

  plt.figure(figsize=(16,9))
  plt.title(name + ' - Schicht: Durchschnitt', fontdict={'fontsize': 25}, loc='left')
  ax = sns.heatmap(df1, annot=True, cmap="YlGnBu", annot_kws={"fontsize":17})

  ax._axes.set_xticklabels(ax._axes.get_xmajorticklabels(), fontsize = 16)
  ax._axes.set_yticklabels(ax._axes.get_ymajorticklabels(), fontsize = 16)
  
  ax.collections[0].colorbar.ax.tick_params(labelsize=16)

  plt.xticks(rotation=30)

  plt.ylabel('Kategorie', fontsize=18)
  plt.xlabel('Algorithmus', fontsize=18)
  ax.yaxis.set_label_position("right")

  folder = name.split(' - ')[0].replace(' ', '_').lower()
  folder_path = f'drive/MyDrive/data/efficientnet_graphics/{folder}/per_layer'
  if not os.path.isdir(folder_path):
    os.mkdir(folder_path)

  figure_name = 'layer_avg_' + name.replace('-', '').replace(' ', '_').replace('__', '_').lower()
  plt.tight_layout()
  plt.savefig(f'{folder_path}/{figure_name}.png', dpi=400)

In [ ]:
# EfficientNet B3
save_path = 'drive/MyDrive/data/efficientnet_b3_results/model_eval_patchlevel'
name = 'EfficientNet B3 - Patch Level'
plot_figure_per_layer_avg(save_path, name)

save_path = 'drive/MyDrive/data/efficientnet_b3_results/model_eval'
name = 'EfficientNet B3 - Image Level'
plot_figure_per_layer_avg(save_path, name)

# EfficientNet B7
save_path = 'drive/MyDrive/data/efficientnet_b7_results/model_eval_patchlevel'
name = 'EfficientNet B7 - Patch Level'
plot_figure_per_layer_avg(save_path, name)

save_path = 'drive/MyDrive/data/efficientnet_b7_results/model_eval'
name = 'EfficientNet B7 - Image Level'
plot_figure_per_layer_avg(save_path, name)

# Multi Layer EfficientNet B7

In [ ]:
def get_color(colorscale_name, loc):
    from _plotly_utils.basevalidators import ColorscaleValidator
    # first parameter: Name of the property being validated
    # second parameter: a string, doesn't really matter in our use case
    cv = ColorscaleValidator("colorscale", "")
    # colorscale will be a list of lists: [[loc1, "rgb1"], [loc2, "rgb2"], ...] 
    colorscale = cv.validate_coerce(colorscale_name)
    
    if hasattr(loc, "__iter__"):
        return [get_continuous_color(colorscale, x) for x in loc]
    return get_continuous_color(colorscale, loc)
        

# Identical to Adam's answer
import plotly.colors
from PIL import ImageColor

def get_continuous_color(colorscale, intermed):
    """
    Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
    color for any value in that range.

    Plotly doesn't make the colorscales directly accessible in a common format.
    Some are ready to use:
    
        colorscale = plotly.colors.PLOTLY_SCALES["Greens"]

    Others are just swatches that need to be constructed into a colorscale:

        viridis_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Viridis)
        colorscale = plotly.colors.make_colorscale(viridis_colors, scale=scale)

    :param colorscale: A plotly continuous colorscale defined with RGB string colors.
    :param intermed: value in the range [0, 1]
    :return: color in rgb string format
    :rtype: str
    """
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    hex_to_rgb = lambda c: "rgb" + str(ImageColor.getcolor(c, "RGB"))

    if intermed <= 0 or len(colorscale) == 1:
        c = colorscale[0][1]
        return c if c[0] != "#" else hex_to_rgb(c)
    if intermed >= 1:
        c = colorscale[-1][1]
        return c if c[0] != "#" else hex_to_rgb(c)

    for cutoff, color in colorscale:
        if intermed > cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    if (low_color[0] == "#") or (high_color[0] == "#"):
        # some color scale names (such as cividis) returns:
        # [[loc1, "hex1"], [loc2, "hex2"], ...]
        low_color = hex_to_rgb(low_color)
        high_color = hex_to_rgb(high_color)

    return plotly.colors.find_intermediate_color(
        lowcolor=low_color,
        highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb",
    )

In [ ]:
import plotly.express as px

_to_test = [
            'bottle', 'cable', 'capsule',
            'carpet', 'grid', 'hazelnut',
            'leather', 'metalnut', 'pill',
            'screw', 'tile', 'toothbrush',
            'transistor', 'wood', 'zipper'
            ]

def extract_results(save_path):
  results = {}
  for category in tqdm(_to_test):
    if 'b3' in save_path:
      end = 8
    elif 'b7' in save_path:
      end = 9
    else:
      print(save_path, 'NOT FOUND')
      break

    results[category] = {}
    layer_combis = [["3", "4"],
                    ["4", "5"],
                    ["3", "5"],
                    ["3", "4", "5"]]
    for layer_combi in layer_combis:

      # create results folder
      
      results_save_path = f'{save_path}/{category}/results_layer_{layer_combi}.pickle'
      try:
        with open(results_save_path, 'rb') as f:
          p = pickle.load(f)
          results[category]["_".join(layer_combi)] = p 
      except:
        continue
  return results

def plot_abod_patch_mean(save_path):
  results = extract_results(save_path)
  
  results_flattened = []
  for category, results_i in results.items():
    for layer_combi, algo_results in results_i.items():
      for algorithm, agg_results in algo_results.items():
        for aggregation, score in agg_results.items():
          d = {
              'Kategorie': category,
              'Schichten': layer_combi,
              'Algorithmus': algorithm.split('_')[0],
              'n_neighbors': algorithm.split('_')[1],
              'Aggregation': '10-mean' if 'top10' in aggregation else 'max',
              'AUC-ROC-Wert': score
          }
          results_flattened.append(d)
  df = pd.DataFrame(results_flattened)

  df_mean = df[['Schichten', 'Algorithmus', 'n_neighbors', 'Aggregation', 'AUC-ROC-Wert']].groupby(
      ['Schichten', 'Algorithmus', 'n_neighbors', 'Aggregation']).mean().reset_index()
  
  def plot_abod_patch(df, title):
    fig = px.bar(df, x="Schichten", y="AUC-ROC-Wert", color="n_neighbors", barmode="group", facet_col="Algorithmus",
                category_orders={"n_neighbors": ["1", "5", "10", "15", "20", "25", "30"],
                                  "Algorithmus": ["ABOD", "PATCH"],
                                  "Schichten": ["3_4", "3_5", "4_5", "3_4_5"]},
                labels={"n_neighbors": "Anzahl<br>Nachbarn"},
                #color_discrete_sequence=get_color("YlGnBu", np.linspace(0, 1, len(df.n_neighbors.unique()))),
                color_discrete_sequence=px.colors.sequential.YlGnBu[1:9], 
                range_y=[0.9, 1],
                title=title,
                width=1200,
                height=500)
    fig.update_layout(
        font=dict(
            size=22
        )
    )
    fig.show()
    img_name = title.lower().replace('.', '_').replace('-', '_').replace(' ', '_')
    fig.write_image(f"drive/MyDrive/data/efficientnet_graphics/fine_tuning/eb7_patch/{img_name}.png",
                    scale=2)

  df_mean1 = df_mean[df_mean['Aggregation'] == 'max']
  df_mean2 = df_mean[df_mean['Aggregation'] == '10-mean']

  plot_abod_patch(df_mean1, title="Max. Anomaliescore")
  plot_abod_patch(df_mean2, title="Avg. Top10-Anomaliescores")


In [ ]:
save_path = 'drive/MyDrive/data/efficientnet_b7_results/model_eval_patchlevel_multi'
plot_abod_patch_mean(save_path)

In [ ]:
import plotly.express as px

_to_test = [
            'bottle', 'cable', 'capsule',
            'carpet', 'grid', 'hazelnut',
            'leather', 'metalnut', 'pill',
            'screw', 'tile', 'toothbrush',
            'transistor', 'wood', 'zipper'
            ]

def extract_results(save_path):
  results = {}
  for category in tqdm(_to_test):
    if 'b3' in save_path:
      end = 8
    elif 'b7' in save_path:
      end = 9
    else:
      print(save_path, 'NOT FOUND')
      break

    results[category] = {}
    layer_combis = [["3", "4", "5"]]
    for layer_combi in layer_combis:

      # create results folder
      
      results_save_path = f'{save_path}/{category}/results_layer_{layer_combi}.pickle'
      try:
        with open(results_save_path, 'rb') as f:
          p = pickle.load(f)
          results[category]["_".join(layer_combi)] = p 
      except:
        continue
  return results

def plot_abod_patch_single(df, title):
  
  fig = px.line(df, x='Kategorie', y='AUC-ROC-Wert', color="Algorithmus", line_dash='Aggregation',
                color_discrete_sequence=[px.colors.sequential.YlGnBu[-6], px.colors.sequential.YlGnBu[-3], '#82083c'],
                category_orders={"Aggregation": ["10-mean", "max"],
                                  "Algorithmus": ["PATCH_n1", "ABOD_n20", "WideResnet_n20"]
                                 },
                width=1200,
                height=500,
                #range_y=[0.7, 1],
                title='')#title)
  fig.update_layout(
      font=dict(
          size=20
      ),
      legend=dict(
          font_size=20,
          orientation="h",
          yanchor="bottom",
          y=1.02,
          xanchor="left",
          x=0
      )
    )
  fig.show()
  img_name = title.replace(':', '_').replace('+', '_').replace(' ', '')
  fig.write_image(f"drive/MyDrive/data/efficientnet_graphics/fine_tuning/eb7_patch/{img_name}.png",
                    scale=2)


def plot_abod_patch(save_path):
  results = extract_results(save_path)

  results_save_path = f'drive/MyDrive/data/wideresnet_evaluation/results_patchcore.pickle'

  with open(results_save_path, 'rb') as f:
    p = pickle.load(f)
  print(p)
  results_flattened = []
  for category, results_i in results.items():
    for layer_combi, algo_results in results_i.items():
      for algorithm, agg_results in algo_results.items():
        for aggregation, score in agg_results.items():
          d = {
              'Kategorie': category,
              'Schichten': layer_combi,
              'Algorithmus': algorithm.split('_')[0],
              'n_neighbors': algorithm.split('_')[1],
              'Aggregation': '10-mean' if 'top10' in aggregation else 'max',
              'AUC-ROC-Wert': score
          }
          results_flattened.append(d)
    ccc = category if category != 'metalnut' else 'metal_nut'
    d = {
              'Kategorie': category,
              'Algorithmus': 'WideResnet_n20',
              'n_neighbors': 'n_20',
              'Aggregation': '',
              'AUC-ROC-Wert': p[ccc][20]
          }
    results_flattened.append(d)

  df = pd.DataFrame(results_flattened)
  df1 = df[(df['Algorithmus'] == 'ABOD') & (df['n_neighbors']=="20") ]
  df1['Algorithmus'] = df1['Algorithmus'] + '_n20'
  df2 = df[(df['Algorithmus'] == 'PATCH') & (df['n_neighbors']=="1") ]
  df2['Algorithmus'] = df2['Algorithmus'] + '_n1'

  df3 = df[df['Algorithmus'] == 'WideResnet_n20']
  
  df = pd.concat([df1, df2, df3], axis=0, ignore_index=True)

  _dfs = []
  cat = df1.sort_values('AUC-ROC-Wert', ascending=False)['Kategorie'].unique().tolist()
  
  for i in cat:
    _dfs.append(df[df['Kategorie'] == i])
  df = pd.concat(_dfs, axis=0, ignore_index=True)

  plot_abod_patch_single(df, title="Schichtkombination: 3 + 4 + 5")


In [ ]:
save_path = 'drive/MyDrive/data/efficientnet_b7_results/model_eval_patchlevel_multi'
plot_abod_patch(save_path)

# Multilayer EfficientNet B3 

In [ ]:
import plotly.express as px
_to_test = [
            'bottle', 'cable', 'capsule',
            'carpet', 'grid', 'hazelnut',
            'leather', 'metalnut', 'pill',
            'screw', 'tile', 'toothbrush',
            'transistor', 'wood', 'zipper'
            ]

def extract_results(save_path):
  results = {}
  for category in tqdm(_to_test):

    results[category] = {}
    layer_combis = [["5", "6"],
                    ["6", "7"],
                    ["5", "7"],
                    ["5", "6", "7"]]
    for layer_combi in layer_combis:

      # create results folder
      
      results_save_path = f'{save_path}/{category}/results_layer_{layer_combi}.pickle'
      with open(results_save_path, 'rb') as f:
        p = pickle.load(f)
        results[category]["_".join(layer_combi)] = p 

  return results

def plot_ae_img(df, title):
  fig = px.bar(df, x="Schichten", y="AUC-ROC-Wert", color="AutoEncoder_layers", barmode="group", facet_col="dropout",
              category_orders={"AutoEncoder_layers": ['512_264_64',
                                                      '1024_512_264_64',
                                                      '2048_1024_512_264',
                                                      '2048_1024_512_264_64',
                                                      '4096_2048_1024_512_264',
                                                      '4096_2048_1024_512_264_64'],
                               "Schichten": ["5_6", "5_7", "6_7", "5_6_7"]},
              labels={"AutoEncoder_layers": "Schichten des AutoEncoders"},
              #color_discrete_sequence=get_color("YlGnBu", np.linspace(0, 1, len(df.n_neighbors.unique()))),
              color_discrete_sequence=px.colors.sequential.YlGnBu[1:9], 
              range_y=[0.8, 1],
              width=1200,
              height=500,
              title=title)
  fig.update_layout(
      font=dict(
          size=22
      )
  )
  fig.show()
  img_name = title.lower().replace(' ', '_')
  fig.write_image(f"drive/MyDrive/data/efficientnet_graphics/fine_tuning/eb3_img/{img_name}.png",
                    scale=3)

def plot_ae_img_mean(save_path):
  results = extract_results(save_path)

  results_flattened = []
  for category, results_i in results.items():
    for layer_combi, autoencoder_results in results_i.items():
      for ae_layers, dropout_results in autoencoder_results.items():
        for droupout, score in dropout_results.items():
          d = {
              'Kategorie': category,
              'Schichten': layer_combi,
              'AutoEncoder_layers': ae_layers,
              'dropout': droupout,
              'AUC-ROC-Wert': score[150]
          }
          results_flattened.append(d)
  df = pd.DataFrame(results_flattened)

  df_mean = df[['Schichten', 'AutoEncoder_layers', 'dropout', 'AUC-ROC-Wert']].groupby(
              ['Schichten', 'AutoEncoder_layers', 'dropout']).mean().reset_index()
  plot_ae_img(df_mean, title="Autoencoder Optimierung")

In [ ]:

save_path = 'drive/MyDrive/data/efficientnet_b3_results/model_eval_imglevel_multi'
plot_ae_img_mean(save_path)

In [ ]:
def plot_ae_img_single(df, title):
  
  fig = px.line(df, x='Kategorie', y='AUC-ROC-Wert', color="Schichten", line_dash='dropout',
                color_discrete_sequence=[px.colors.sequential.YlGnBu[-6], px.colors.sequential.YlGnBu[-4], px.colors.sequential.YlGnBu[-1]],
                category_orders={"Kategorie": ['leather', 'bottle', 'carpet','wood','tile','zipper',
                                               'hazelnut','cable', 'transistor','toothbrush','grid',
                                               'metalnut','capsule','pill','screw'],
                                 "dropout": [0.1, 0.2]
                                 },
                width=1200,
                height=500,
                #range_y=[0.7, 1],
                title='')#title)
  fig.update_layout(legend=dict(
      orientation="h",
      yanchor="bottom",
      y=1.02,
      xanchor="right",
      x=1
  ),
  font=dict(
          size=20
  ))
  fig.show()
  img_name = title.lower().replace('die ', '').replace(' ', '_')
  fig.write_image(f"drive/MyDrive/data/efficientnet_graphics/fine_tuning/eb3_img/{img_name}.png",
                    scale=3)

def extract_results(save_path):
  results = {}
  for category in tqdm(_to_test):

    results[category] = {}
    layer_combis = [["5", "6"],
                    ["6", "7"],
                    ["5", "7"],
                    ["5", "6", "7"]]
    for layer_combi in layer_combis:

      # create results folder
      
      results_save_path = f'{save_path}/{category}/results_layer_{layer_combi}.pickle'
      with open(results_save_path, 'rb') as f:
        p = pickle.load(f)
        results[category]["_".join(layer_combi)] = p 

  return results
  
def plot_ae_img_select(save_path):
  results = extract_results(save_path)

  results_flattened = []
  for category, results_i in results.items():
    for layer_combi, autoencoder_results in results_i.items():
      for ae_layers, dropout_results in autoencoder_results.items():
        for droupout, score in dropout_results.items():
          d = {
              'Kategorie': category,
              'Schichten': layer_combi,
              'AutoEncoder_layers': ae_layers,
              'dropout': droupout,
              'AUC-ROC-Wert': score[150]
          }
          results_flattened.append(d)
  df = pd.DataFrame(results_flattened)
  

  
  df1 = df[(df['Schichten']=="6_7") & (df['AutoEncoder_layers'] == '512_264_64') & (df['dropout']==0.1)]
  df2 = df[(df['Schichten']=="5_6_7") & (df['AutoEncoder_layers'] == '512_264_64') & (df['dropout']==0.1)]
  df3 = df[(df['Schichten']=="5_6_7") & (df['AutoEncoder_layers'] == '2048_1024_512_264') & (df['dropout']==0.2)]
  print(df2.sort_values('AUC-ROC-Wert', ascending=False)['Kategorie'].unique())

  df = pd.concat([df3, df1, df2], axis=0, ignore_index=True)
  df["Schichten"] = 'EN: ' + df["Schichten"] + ' - AE: ' + df["AutoEncoder_layers"]
  _dfs = []
  for i in ['leather', 'bottle', 'carpet','wood','tile','zipper','hazelnut','cable', 'transistor','toothbrush','grid','metalnut','capsule','pill','screw']:

    _dfs.append(df[df['Kategorie'] == i])
  df = pd.concat(_dfs, axis=0, ignore_index=True)

  plot_ae_img_single(df, title="Die besten AutoEncoder Kombinationen")


In [ ]:

save_path = 'drive/MyDrive/data/efficientnet_b3_results/model_eval_imglevel_multi'
plot_ae_img_select(save_path)

# Wide Resnet Comparison

In [ ]:
results_save_path = f'drive/MyDrive/data/wideresnet_evaluation/patch_vs_abod'
_to_test = [
            'bottle', 'cable', 'capsule',
            'carpet', 'grid', 'hazelnut',
            'leather', 'metal_nut', 'pill',
            'screw', 'tile', 'toothbrush',
            'transistor', 'wood', 'zipper'
            ]
complete_results = []
for i in _to_test:
  with open(f'{results_save_path}/{i}.pickle', 'rb') as f:
    results = pickle.load(f)
  for algo, algo_res in results.items():
    for aggregation, score in algo_res.items():
      complete_results.append({'Algorithmus': algo.split('_')[0],
                               'Anzahl Nachbarn': algo.split('_')[1],
                               'Aggregation': 'Max' if aggregation.split('_')[-1] == 'top1' else aggregation.split('_')[-1].capitalize(),
                               'AUC-ROC Wert': score})
df = pd.DataFrame(complete_results)
df_mean = df.groupby(by=['Algorithmus', 'Anzahl Nachbarn', 'Aggregation']).mean().reset_index()
df_mean['Anzahl Nachbarn'] = df_mean['Anzahl Nachbarn'].astype(int)
df_mean.sort_values(['Anzahl Nachbarn'], ascending=True, inplace=True)


In [ ]:

fig = px.line(df_mean,
              x='Anzahl Nachbarn',
              y='AUC-ROC Wert',
              color='Algorithmus',
              line_dash='Aggregation',
              color_discrete_sequence=[px.colors.sequential.YlGnBu[-4], '#c02058'],
              category_orders={'Anzahl Nachbarn': [1, 5, 10, 15, 20, 25, 30]},
              width=1200,
              height=500,
              #line_shape='spline',
              title='ABOD vs. Patch (WideResNet)')
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
),
  font=dict(
          size=20
  ))
fig.show()
img_name = 'ABOD vs. Patch (WideResNet'.lower().replace(' ', '_')
fig.write_image(f"drive/MyDrive/data/wideresnet_evaluation/{img_name}.png",
                scale=3)
#img_name = title.lower().replace('die ', '').replace(' ', '_')


In [ ]:
results_save_path = f'drive/MyDrive/data/wideresnet_evaluation/patch_vs_abod'
_to_test = [
            'bottle', 'cable', 'capsule',
            'carpet', 'grid', 'hazelnut',
            'leather', 'metal_nut', 'pill',
            'screw', 'tile', 'toothbrush',
            'transistor', 'wood', 'zipper'
            ]
results_flatten = []
for i in _to_test:
  with open(f'{results_save_path}/{i}.pickle', 'rb') as f:
    results = pickle.load(f)
  results_flatten.append({'Kategorie': i,
                          'PATCH': results['PATCH_1']['img_auc_top1'],
                          'ABOD': results['ABOD_10']['img_auc_top1']})
  
df = pd.DataFrame(results_flatten)#

df['Differenz'] = df['ABOD'] - df['PATCH']
df = df.set_index('Kategorie').T
df.sort_values(axis=1, by='PATCH', inplace=True, ascending=False)
df['mean'] = df.mean(axis=1)
plt.figure(figsize=(20,7))

# scale the norm
df_plot = df.copy(deep=True)
df_plot.loc['Differenz', 'screw'] = -0.018
for i in range(df_plot.shape[0]):
  df_plot.iloc[i, :] = (df_plot.iloc[i, :] - df_plot.iloc[i, :].min())/(df_plot.iloc[i, :].max() - df_plot.iloc[i, :].min())

sns.set(font_scale=1.2)
g = sns.heatmap(df_plot, annot=df, cmap="RdYlGn", fmt='.3f', linewidths=1, annot_kws={"fontsize":18})

g._axes.set_xticklabels(g._axes.get_xmajorticklabels(), fontsize = 20)
g._axes.set_yticklabels(g._axes.get_ymajorticklabels(), fontsize = 20)
g.collections[0].colorbar.ax.tick_params(labelsize=20)
plt.xticks(rotation=30)

#figure_name = 'complete_' + name.replace('-', '').replace(' ', '_').replace('__', '_').lower()
plt.tight_layout()
plt.savefig(f'drive/MyDrive/data/wideresnet_evaluation/wrsn_abod_patch.png', dpi=400)
plt.show()

# Wide ResNet standalone

In [ ]:
results_save_path = f'drive/MyDrive/data/wideresnet_evaluation/patch_vs_abod'
_to_test = [
            'bottle', 'cable', 'capsule',
            'carpet', 'grid', 'hazelnut',
            'leather', 'metal_nut', 'pill',
            'screw', 'tile', 'toothbrush',
            'transistor', 'wood', 'zipper'
            ]

results_flatten = []
for i in _to_test:
  with open(f'{results_save_path}/{i}.pickle', 'rb') as f:
    results = pickle.load(f)
  results_flatten.append({'Kategorie': i,
                          'PATCH': results['PATCH_1']['img_auc_top1']})
  
df = pd.DataFrame(results_flatten)#
#df['Kategorie'] = df['Kategorie'].apply(lambda x: rename_dict[x])
df['Kategorie'] = df['Kategorie'].apply(lambda x: 'metalnut' if x == 'metal_nut' else x)
df = df.set_index('Kategorie').T
df.sort_values(axis=1, by='PATCH', inplace=True, ascending=False)
df['mean'] = df.mean(axis=1)
plt.figure(figsize=(20,3))


sns.set(font_scale=1.5)
g = sns.heatmap(df, annot=df, cmap="YlGnBu", fmt='.3g', linewidths=1)

g._axes.set_xticklabels(g._axes.get_xmajorticklabels(), fontsize = 20)
g._axes.set_yticklabels(g._axes.get_ymajorticklabels(), fontsize = 20)
plt.xticks(rotation=30)

#figure_name = 'complete_' + name.replace('-', '').replace(' ', '_').replace('__', '_').lower()
plt.tight_layout()
plt.savefig(f'drive/MyDrive/data/wideresnet_evaluation/wrsn.png', dpi=400)
plt.show()